In [ ]:
from fludashboard import settings
from fludashboard.libs import episem
from numpy import ceil
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import fludashboard as flud
import pandas as pd
import sqlalchemy as sqla
import cufflinks as cf

init_notebook_mode(connected=True)
dsn = 'postgresql://%(USER)s:%(PASSWORD)s@%(HOST)s/%(NAME)s'
engine = sqla.create_engine(dsn % settings.DATABASE)

cf.set_config_file(offline=True)
cf.set_config_file(theme='white')

<script>requirejs.config({paths: { 'plotly': ['https://cdn.plot.ly/plotly-latest.min']},});if(!window.Plotly) {{require(['plotly'],function(plotly) {window.Plotly=plotly;});}}</script>

In [30]:

def opportunities_data(year=int, territory_type_id=int, week=None, territory_id=None):
    '''
    Grab data for opportunity boxplots
    
    :param year: selected year
    :param week: selected week. 0 or Nonoe for all
    :param territory_id: territory. 0 or None for whole country
    :param territory_type_id: territory type. None or 4 for whole country 
    :return pd.DataFrame: 
    '''
    
    sql_param = {
        'epiweek': week,
        'epiyear': year,
        'territory_id': territory_id,
        'territory_select': '',
        'territory_name': '',
        'territory_join': ''
    }
    
    territory_type2column = {
        1: 'territory_id',
        2: 'regional',
        3: 'region'
    }
    
    if territory_id not in [0, None]:
        sql_param['territory_name'] = ', territory.name AS territory_name'
        sql_param['territory_column'] = territory_type2column[territory_type_id] 
        sql_param['territory_select'] = '''AND %(territory_column)s = %(territory_id)s''' % sql_param
        sql_param['territory_join'] = '''LEFT JOIN territory
            ON (delay.%(territory_column)s=territory.id)
        WHERE 1=1
        ''' % sql_param
    if week == None or week == 0:
        sql_param['epiweek'] = 53
    
    sql = '''
    SELECT
        delay.notification2digitalization AS "Notificação à digitalização",
        delay.symptoms2notification as "Primeiros sintomas à notificação",
        delay.symptoms2digitalization AS "Primeiros sintomas à notificação",
        delay.symptoms2antiviral AS "Primeiros sintomas ao tratamento",
        delay.symptoms2sample AS "Primeiros sintomas à coleta",
        delay.sample2ifi AS "Coleta a resultado de IFI",
        delay.sample2PCR AS "Coleta a resultado de PCR",
        delay.notification2closure AS "Notificação ao encerramento"        
        %(territory_name)s
    FROM
        (SELECT * 
        FROM
            delay_table
        WHERE
            epiyear=%(epiyear)s
            AND dataset_id=1
            AND epiweek <= %(epiweek)s
            %(territory_select)s
        ) as delay
        %(territory_join)s
    ''' % sql_param

    print(sql)
    with engine.connect() as conn:
        df = pd.read_sql(sql, conn)
    
    if territory_id is None:
        df['territory_name'] = 'Brasil'
    return df

<script>requirejs.config({paths: { 'plotly': ['https://cdn.plot.ly/plotly-latest.min']},});if(!window.Plotly) {{require(['plotly'],function(plotly) {window.Plotly=plotly;});}}</script>

In [ ]:
df = opportunities_data(year=2017, territory_id=43, territory_type_id=1)
print(df.head())


    SELECT
        delay.notification2digitalization AS "Notificação à digitalização",
        delay.symptoms2notification as "Primeiros sintomas à notificação",
        delay.symptoms2digitalization AS "Primeiros sintomas à notificação",
        delay.symptoms2antiviral AS "Primeiros sintomas ao tratamento",
        delay.symptoms2sample AS "Primeiros sintomas à coleta",
        delay.sample2ifi AS "Coleta a resultado de IFI",
        delay.sample2PCR AS "Coleta a resultado de PCR",
        delay.notification2closure AS "Notificação ao encerramento"        
        , territory.name AS territory_name
    FROM
        (SELECT * 
        FROM
            delay_table
        WHERE
            epiyear=2017
            AND dataset_id=1
            AND epiweek <= 53
            AND territory_id = 43
        ) as delay
        LEFT JOIN territory
            ON (delay.territory_id=territory.id)
        WHERE 1=1
        
    
   Notificação à digitalização  Primeiros sintomas à notificação  

<div id="24051c2b-ebd7-4177-8b99-1d206c0e035b" style="height: 525px; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">require(["plotly"], function(Plotly) { window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("24051c2b-ebd7-4177-8b99-1d206c0e035b", [{"type": "box", "y": [2, 0, 0, 0, 1, 0, 3, 3, 0, 0, 1, 2, 1, 1, 1, 0, 0, 0, 1, 2, 0, 0, 2, 7, 3, 0, 6, 0, 0, 0, 3, 4, 2, 0, 1, 17, 5, 0, 0, 6, 2, 0, 0, 3, 0, 4, 3, 1, 0, 1, 0, 20, 1, 2, 2, 0, 0, 0, 2, 3, 14, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 9, 29, 2, 4, 1, 0, 5, 2, 2, 2, 7, 0, 2, 2, 2, 0, 0, 0, 1, 3, 4, 1, 1, 5, 5, 7, 4, 1, 2, 1, 1, 0, 0, 0, 1, 1, 6, 6, 155, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 3, 5, 0, 0, 1, 6, 0, 0, 1, 1, 1, 0, 0, 0, 4, 0, 0, 3, 0, 0, 0, 3, 0, 4, 0, 2, 1, 2, 2, 0, 15, 1, 1, 11, 3, 5, 2, 0, 2, 2, 1, 0, 0, 3, 3, 3, 0, 4, 4, 1, 1, 1, 1, 1, 5, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 4, 4, 0, 0, 4, 0, 3, 6, 1, 1, 1, 1, 1, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 4, 1, 1, 2, 1, 3, 4, 1, 4, 6, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 1, 2, 2, 0, 3, 3, 3, 2, 2, 0, 1, 2, 0, 6, 3, 3, 3, 3, 3, 0, 1, 8, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 1, 1, 8, 6, 4, 13, 2, 1, 3, 0, 3, 4, 5, 0, 4, 4, 4, 0, 0, 18, 18, 0, 18, 0, 1, 1, 18, 18, 18, 18, 0, 0, 0, 9, 2, 3, 2, 1, 0, 2, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 1, 4, 1, 5, 0, 0, 0, 1, 1, 0, 0, 1, 1, 5, 1, 1, 1, 2, 2, 13, 1, 0, 0, 3, 5, 1, 0, 5, 5, 5, 4, 7, 4, 6, 34, 34, 11, 6, 8, 55, 116, 0, 0, 1, 1, 1, 1, 2, 4, 1, 2, 8, 2, 3, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 1, 3, 2, 3, 24, 3, 1, 0, 0, 3, 4, 1, 0, 1, 0, 1, 2, 3, 1, 0, 0, 0, 1, 1, 2, 3, 2, 3, 4, 4, 4, 5, 7, 4, 1, 5, 5, 4, 2, 2, 4, 2, 3, 3, 1, 4, 1, 1, 1, 8, 3, 8, 5, 1, 3, 0, 2, 0, 2, 3, 2, 5, 1, 4, 3, 3, 2, 1, 2, 2, 1, 1, 5, 1, 3, 4, 3, 10, 10, 1, 0, 0, 0, 1, 3, 1, 2, 2, 1, 0, 1, 3, 3, 0, 8, 0, 4, 0, 1, 3, 1, 2, 3, 3, 3, 2, 1, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 7, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 3, 4, 0, 0, 3, 0, 3, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 3, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 4, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 3, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 3, 3, 0, 3, 0, 0, 3, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 7, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 4, 1, 1, 1, 1, 1, 1, 5, 1, 1, 2, 2, 1, 0, 0, 1, 1, 1, 3, 2, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 3, 27, 53, 60, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 6, 7, 8, 1, 1, 5, 4, 4, 8, 6, 2, 2, 2, 4, 5, 2, 2, 1, 4, 5, 6, 0, 1, 2, 3, 0, 0, 1, 9, 1, 10, 1, 0, 0, 0, 1, 0, 0, 0, 9, 8, 3, 1, 0, 1, 0, 0, 8, 8, 1, 0, 5, 4, 7, 0, 10, 1, 1, 4, 2, 2, 2, 5, 8, 1, 22, 19, 22, 11, 14, 0, 0, 0, 0, 4, 3, 1, 1, 4, 0, 0, 1, 0, 1, 1, 6, 3, 2, 3, 3, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 7, 4, 1, 8, 0, 1, 1, 7, 1, 3, 0, 1, 2, 2, 2, 2, 1, 0, 19, 5, 4, 4, 0, 0, 0, 0, 13, 4, 4, 4, 4, 4, 4, 4, 4, 3, 11, 11, 11, 4, 4, 4, 4, 6, 4, 4, 7, 9, 1, 10, 3, 13, 0, 12, 1, 8, 1, 0, 0, 3, 0, 11, 21, 0, 11, 0, 1, 0, 1, 0, 10, 15, 13, 6, 13, 3, 2, 0, 2, 3, 2, 1, 1, 0, 2, 2, 4, 9, 1, 1, 0, 2, 2, 2, 0, 0, 1, 2, 4, 3, 3, 3, 3, 0, 4, 3, 1, 0, 0, 1, 1, 4, 0, 0, 0, 0, 4, 1, 2, 1, 1, 3, 3, 3, 1, 4, 2, 2, 2, 4, 4, 4, 1, 2, 2, 1, 1, 2, 5, 1, 1, 2, 6, 6, 6, 6, 2, 2, 1, 4, 2, 2, 2, 2, 5, 4, 4, 2, 4, 4, 2, 1, 1, 2, 3, 3, 69, 69, 1

In [29]:
df = opportunities_data(year=2017, week=1, territory_type_id=1)
print(df.head())


    SELECT
        delay.epiweek AS epiweek,
        delay.notification2digitalization AS "Notificação à digitalização",
        delay.symptoms2notification as "Primeiros sintomas à notificação",
        delay.symptoms2digitalization AS "Primeiros sintomas à notificação",
        delay.symptoms2antiviral AS "Primeiros sintomas ao tratamento",
        delay.symptoms2sample AS "Primeiros sintomas à coleta",
        delay.sample2ifi AS "Coleta a resultado de IFI",
        delay.sample2PCR AS "Coleta a resultado de PCR",
        delay.notification2closure AS "Notificação ao encerramento"        
        
    FROM
        (SELECT * 
        FROM
            delay_table
        WHERE
            epiyear=2017
            AND dataset_id=1
            AND epiweek <= 1
            
        ) as delay
        
    
   epiweek  Notificação à digitalização  Primeiros sintomas à notificação  \
0        1                            0                                 3   
1        1                   